***
## Выборка по столбцам

Одно из наиболее используемых ключевых слов в SQL — `SELECT` (англ. «выбрать»). Эта инструкция позволяет запрашивать данные из таблиц и при необходимости приводить полученные результаты к нужному виду.

После ключевого слова `SELECT` в запросе указывают названия полей, значения которых должны вернуться в ответе; после `FROM` (англ. «из») — названия таблиц, где надо искать данные.

```sql
SELECT <перечень столбцов, значения из которых надо получить>
FROM <перечень таблиц, из которых надо получить данные>;
```

В ответ на запрос `SELECT` данные возвращаются в структурированном табличном виде. К такой таблице можно снова применить запрос `SELECT`, то есть запросы могут быть «вложены» друг в друга, а их результаты — объединены. Вложенные запросы называют подзапросами.

Чтобы не путаться в понятиях «таблица БД» и «таблица с ответом», возвращаемые данные будем называть «результирующая выборка» или просто «выборка».

В ответ на запрос 

```sql
-- Вернуть только поля title, release_year
SELECT title,
       release_year 
-- из всех записей в таблице video_products
FROM video_products;
```

Python «под капотом» превращает результирующую выборку в итерируемый объект, который можно перебрать в цикле; элементами этого объекта будут кортежи со значениями полей, полученных из БД. 

Прочитаем и выведем в консоль все поля всех записей из таблицы **video_products**:

In [ ]:
import sqlite3

con = sqlite3.connect('db_video.sqlite')
cur = con.cursor()

# Запрашиваем все столбцы всех записей из таблицы video_products;
# символ * после SELECT означает "верни все поля найденных записей".
results = cur.execute('''
  SELECT *
  FROM video_products;
''')

print(results)

# В results получим итерируемый объект, который можно перебрать циклом:
for result in results:
    print(result)

# При получении данных из таблицы не нужно вызывать метод con.commit()!
con.close()

(1, 'Весёлые мелодии', 'Мультсериал', 1930)
(2, 'Безумные Мелодии Луни Тюнз', 'Мультсериал', 1931)
(3, 'Кто подставил кролика Роджера', 'Фильм', 1988)
(4, 'Хороший, плохой, злой', 'Фильм', 1967)
(5, 'Последний киногерой', 'Фильм', 1993)
(6, 'Она написала убийство', 'Сериал', 1984)
(7, 'Лас-Вегас', 'Сериал', 2003)
(8, 'Паркер Льюис не проигрывает', 'Сериал', 1990)
(9, 'Розовая пантера: Контроль за вредителями', 'Мультфильм', 1969)
(10, 'Койот против Acme', 'Фильм', 2023)


Обычно при запросах к БД разработчику не нужны все поля записей. Например, на странице сайта требуется показать только названия и год выпуска всех фильмов из базы данных. В этой ситуации лучше запросить из базы не все поля, а лишь два из них; названия столбцов перечисляют через запятую после `SELECT`. 

Поля будут выгружены из таблицы в том порядке, как они указаны в запросе. 

```py
...

results = cur.execute('''
    SELECT title,
           release_year
    FROM video_products;
''')

...
```

Если теперь перебрать в цикле и распечатать содержимое `results` — будут напечатаны десять кортежей, но в каждом кортеже будет только два элемента: название фильма и год выпуска.

```py
('Весёлые мелодии', 1930)
('Безумные мелодии Луни Тюнз', 1931)
('Кто подставил кролика Роджера', 1988)
('Хороший, плохой, злой', 1967)
('Последний киногерой', 1993)
('Она написала убийство', 1984)
('Лас-Вегас', 2003)
('Паркер Льюис не проигрывает', 1990)
('Розовая пантера: Контроль за вредителями', 1969)
('Койот против Acme', 2023)
```

***
## Выборка по строкам

В запросе можно отфильтровать записи, удовлетворяющие заданным условиям. Найдём все фильмы, снятые после 1980 года. 

Для фильтрации можно применить ключевое слово `WHERE`:

```sql
SELECT title,
       release_year
FROM video_products
-- ГДЕ значение поля release_year больше 1980
WHERE release_year > 1980;
```

![alt text](https://pictures.s3.yandex.net/resources/S02_130_1678062715.png)

В результирующую выборку попадут только те записи, которые соответствуют условиям, описанным после `WHERE`.

Для описания условий запроса применяют специальные операторы — большинство из них вам знакомы. 

Вот несколько популярных операторов:

* `=` — оператор сравнения, а не присваивания (это аналог `==` в Python).

```sql
  SELECT title
  FROM video_products
  -- Найдём фильмы, выпущенные в 1988:
  WHERE release_year = 1988;
```

* `>` , `<` — «больше» и «меньше», `>=` , `<=` — «больше или равно», «меньше или равно».

```sql
  SELECT title
  FROM video_products
  -- Найдём фильмы, выпущенные позже 1988:
  WHERE release_year > 1988;
```

* `<>` — «не равно». В SQLite в качестве оператора неравенства можно применять и `!=`.

```sql
  SELECT title
  FROM video_products
  -- Найдём фильмы, выпущенные в любом году, кроме 1988:
  WHERE release_year <> 1988;
```

* `BETWEEN ... AND ...` — «между», для проверки значения в диапазоне: `BETWEEN начало_диапазона AND конец_диапазона`. **Начало и конец диапазона включены в условие**.

```sql
  SELECT title
  FROM video_products
  -- Найдём фильмы, выпущенные с 1980 по 1990 год включительно:
  WHERE release_year BETWEEN 1980 AND 1990;
```

* `IN` — вхождение в список:

```sql
  SELECT title
  FROM video_products
  -- Найдём в базе все фильмы, 
  -- у которых значение поля product_type - 'Сериал' или 'Фильм':
  WHERE product_type IN ('Сериал', 'Фильм');
```

* `LIKE` — поиск строки по шаблону. В шаблонах можно применять символы-«маски»: знак процента `%` заменяет любой **набор символов**; символ подчёркивания `_` заменяет **один любой символ** (цифру, букву, пробел, пунктуационный или любой другой символ).

```sql
  SELECT title
  FROM video_products
  WHERE product_type LIKE 'Мульт%';
``` 

Этот запрос выберет все записи, где значение поля `product_type` — «Мультфильм» или «Мультсериал». Символы `_` и `%` можно комбинировать в одном шаблоне. 

При сравнении значения поля с числом — всё просто: `WHERE название_поля = 5`.

Но если сравниваются строковые данные, то набор символов берут в одинарные кавычки: `WHERE название_поля = 'Сериал'`.


Есть и другие операторы, их описание можно посмотреть в справочниках, например, [в этом](https://www.sqlite.org/optoverview.html#where_clause_analysis).

Синтаксис SQL-запроса позволяет использовать в `WHERE` множество условий одновременно. Для объединения сразу нескольких условий используются операторы `AND`, `OR` или `NOT`, они работают так же, как в Python.

```py
...

results = cur.execute('''
SELECT title,
       release_year
FROM video_products
WHERE (release_year BETWEEN 1965 AND 1990) AND product_type LIKE '%ильм';
''')

for result in results:
    print(result)
...
```

Если выполнить этот код — вернётся такой результат:

```bash
('Кто подставил кролика Роджера', 1988)
('Хороший, плохой, злой', 1967)
('Розовая пантера: Контроль за вредителями', 1969)
```

Когда используется несколько условий, каждое из них проверяется на истинность, и на выходе получается либо значение `TRUE`, если утверждение истинно, либо `FALSE`, если утверждение ложно. Затем каждый из логических операторов действует по своим правилам:

* `AND` возвращает `TRUE`, только если оба логических значения тоже `TRUE`.

* `OR` возвращает `TRUE`, если хотя бы одно логическое значение тоже `TRUE`.

* `NOT` меняет значение выражения на противоположное: `TRUE` на `FALSE`, а `FALSE` на `TRUE`.

***
## Приоритет

Логические операторы имеют разный приоритет выполнения: оператор `NOT` выполняется в первую очередь, следующим выполняется оператор `AND`; у оператора `OR` — самый низкий приоритет. Для объединения условий в группу используются скобки `()`: операции в скобках, как и в арифметике, выполняются прежде всего.

При выполнении запроса…

```sql
SELECT title,
       product_type,
       release_year
FROM video_products
WHERE product_type = 'Фильм' OR product_type = 'Сериал' AND release_year = 1993;
```

…результат будет такой:

```bash
('Кто подставил кролика Роджера', 'Фильм', 1988)
('Хороший, плохой, злой', 'Фильм', 1967)
('Последний киногерой', 'Фильм', 1993)
('Койот против Acme', 'Фильм', 2023)
```

Изменим приоритеты — возьмём в скобки выражение с оператором OR:

```sql
SELECT title,
       product_type,
       release_year
FROM video_products
WHERE (product_type = 'Фильм' OR product_type = 'Сериал') AND release_year = 1993;
```

Результат станет совершенно иным:


```bash
('Последний киногерой', 'Фильм', 1993)
```

***
## Специальное значение NULL

Значение `NULL` означает отсутствие данных. Основная особенность `NULL` в том, что это значение не равно ничему; любая попытка сравнить `NULL` с чем угодно, даже с `NULL`, вернёт `False`. 

Для работы с `NULL` используют отдельные операторы — `IS NULL` и `IS NOT NULL`. Для значения `NULL` оператор `IS NULL` вернёт `True`; оператор `IS NOT NULL` вернёт `True`, если значение не равно `NULL`.

Операторы `IS NULL` и `IS NOT NULL` можно применять в условиях, чтобы фильтровать данные. 

***
## Получение уникальных значений из столбца

Чтобы получить уникальные значения из заданного столбца — применяют инструкцию `DISTINCT`.

Выберем уникальные значения из колонки `product_type` в таблице `video_products`:

```sql
-- ВЫБРАТЬ УНИКАЛЬНЫЕ ЗНАЧЕНИЯ из колонки product_type:
SELECT DISTINCT product_type
FROM video_products;
```

Получим перечень типов фильмов, без повторов:

```bash
('Мультсериал',)
('Фильм',)
('Сериал',)
('Мультфильм',)
```

In [ ]:
import sqlite3

con = sqlite3.connect('db.sqlite')
cur = con.cursor()

# Напишите SQL запрос в строке.
cur.execute('''
SELECT tbl_name
FROM sqlite_master
WHERE type='table';
''')

table = cur.fetchall()[0][0]  # Получите имя таблицы через атрибут курсора.

# Напишите SQL запрос в строке.
results = cur.execute(f'''
SELECT title,
       description
FROM {table};
''')


for result in results:
    print(result)

con.close()

('Весёлые мелодии', 'Мультсериал')
('Безумные Мелодии Луни Тюнз', 'Мультсериал')
('Кто подставил кролика Роджера', 'Фильм')
('Хороший, плохой, злой', 'Фильм')
('Последний киногерой', 'Фильм')
('Она написала убийство', 'Сериал')
('Лас-Вегас', 'Сериал')
('Паркер Льюис не проигрывает', 'Сериал')
('Розовая пантера: Контроль за вредителями', 'Мультфильм')
('Койот против Acme', 'Фильм')
